In [1]:
import pandas as pd

df = pd.read_csv('../dados/acordaos-unicos-filtrados-500.csv', sep = '|')
df['filtrado_500']=df['filtrado_500'].astype(str)
df.head()

,acordao,areas,filtrado_500
0,297/2016-P,Responsabilidade,conta conta gerência instituto nacional seguro...
1,366/2016-P,Finanças Públicas,senado petróleo petróleo regime aduaneiro expo...
2,944/2016-P,Responsabilidade,tribunal conta plenário embargo declaração aco...
3,30/2016-P,Direito Processual,embargo declaração inss recorrente marco antôn...
4,55/2016-P,Pessoal,senac senac senac empresa senac giselle araújo...


In [2]:
df.shape

(9739, 3)

In [3]:
from sklearn.preprocessing import LabelBinarizer

areas = df.groupby(['areas']).groups.keys()
lbArea = LabelBinarizer()
lbArea.fit([x for x in areas])
lbArea.classes_

array(['Competência do TCU', 'Contrato Administrativo', 'Convênio',
       'Desestatização', 'Direito Processual', 'Finanças Públicas',
       'Gestão Administrativa', 'Licitação', 'Pessoal',
       'Responsabilidade'], dtype='<U23')

In [4]:
y = lbArea.transform(df['areas'])
y.shape

(9739, 10)

In [5]:
from keras.preprocessing.text import Tokenizer
import numpy as np

vocabulario = 35000
limite_texto = 500
dim_vetor = 100

tokenizer = Tokenizer(num_words=vocabulario)
tokenizer.fit_on_texts(df['filtrado_500'])

sequences = tokenizer.texts_to_sequences(df['filtrado_500'])

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Using TensorFlow backend.


Found 32293 unique tokens.


In [6]:
from keras.preprocessing.sequence import pad_sequences

x = pad_sequences(sequences, maxlen=limite_texto)

print('Shape of data tensor:', x.shape)

Shape of data tensor: (9739, 500)


In [7]:
x.shape, y.shape

((9739, 500), (9739, 10))

In [8]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding
from keras.layers.core import Dropout

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, input_length=x.shape[1]))
model.add(Flatten())
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',  metrics=["categorical_accuracy"])
model.summary()

W0312 11:39:35.396537 140273948411712 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0312 11:39:35.430149 140273948411712 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0312 11:39:35.432897 140273948411712 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0312 11:39:35.463115 140273948411712 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is d

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 100)          3500000   
_________________________________________________________________
flatten_1 (Flatten)          (None, 50000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 2048)              102402048 
_________________________________________________________________
dropout_1 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               1049088   
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total para

In [9]:
history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

W0312 11:39:35.607313 140273948411712 deprecation.py:323] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 7791 samples, validate on 1948 samples
Epoch 1/20
7791/7791 [==============================] - 35s 4ms/step - loss: 1.1781 - categorical_accuracy: 0.6051 - val_loss: 1.3475 - val_categorical_accuracy: 0.5246
Epoch 2/20
7791/7791 [==============================] - 30s 4ms/step - loss: 0.4971 - categorical_accuracy: 0.8384 - val_loss: 1.5799 - val_categorical_accuracy: 0.5400
Epoch 3/20
7791/7791 [==============================] - 30s 4ms/step - loss: 0.1130 - categorical_accuracy: 0.9661 - val_loss: 2.0180 - val_categorical_accuracy: 0.5390
Epoch 4/20
7791/7791 [==============================] - 30s 4ms/step - loss: 0.0462 - categorical_accuracy: 0.9867 - val_loss: 2.2743 - val_categorical_accuracy: 0.5431
Epoch 5/20
7791/7791 [==============================] - 30s 4ms/step - loss: 0.0313 - categorical_accuracy: 0.9909 - val_loss: 2.3800 - val_categorical_accuracy: 0.5549
Epoch 6/20
7791/7791 [==============================] - 30s 4ms/step - loss: 0.0271 - categorical_accuracy: